In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pydicom
!pip install opencv-python
!pip install pillow # optional
!pip install pandas
!pip3 install numpy
!pip3 install dicom2nifti
!pip3 install nibabel
!pip3 install pydicom
!pip3 install tqdm
!pip3 install nilearn
!pip install --quiet torchio==0.18.90

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 672.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.7/172.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 11.3 MB/s eta 0:00:00


In [ ]:
import pathlib as plb
import tempfile
import os
import dicom2nifti
import nibabel as nib
import numpy as np
import pydicom
import sys
import shutil
import nilearn.image
from tqdm import tqdm
import cv2

import enum
import time
import random
import multiprocessing
from pathlib import Path

import torch
import torchvision
import torchio as tio
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.utils import save_image

import numpy as np
# from unet import UNet
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

from IPython import display
from tqdm.auto import tqdm
from pathlib import Path

from PIL import Image

from skimage.measure import regionprops_table

In [ ]:
debug = False

In [ ]:
# Processor Class
class TwoDimensionSlicesProcessing():
  def __init__(self,
               nifti_folder: str,
               output_folder: str,
               label: str,
               label_file: str,
               channels: int,
               save_negatives: bool,
               voxel_size: int = 2,
               plane: str = 'axial'
  ):

    self.nifti_folder = plb.Path(nifti_folder)
    self.output_folder = plb.Path(output_folder)

    # make folder
    if not os.path.isdir(self.output_folder):
      os.mkdir(self.output_folder)

    self.label = label
    self.label_file = plb.Path(label_file)

    self.voxel_size = voxel_size
    self.transform = tio.transforms.Resample(self.voxel_size)

    self.channels = channels
    assert self.channels in [1, 3]

    self.save_negatives = save_negatives

    self.plane = plane
    self.plane_to_axis = {
        'axial': 2,     # x-axis
        'coronal': 1,   # y-axis
        'sagittal': 0   # z-axis
    }
    self.axis = self.plane_to_axis.get(self.plane, None)

    assert self.axis is not None

    self.delim = ','


  def find_ct_files(self):
    # find all ct files
    patient_dirs = list(self.nifti_folder.glob('*'))
    print(patient_dirs)
    ct_dirs = []
    ct_files = 'SUV.nii.gz'
    seg_files = 'SEG.nii.gz'

    for dir in patient_dirs:
      # print(dir)
      sub_dirs = list(dir.rglob(ct_files)) + list(dir.rglob(seg_files))
      # print(sub_dirs)
      if len(sub_dirs)==2:
        ct_dirs.append(sub_dirs) # list of lists[2] with matching SUV and SEG files
      else:
        continue

    return ct_dirs


  def load_and_standardize_spacing(self, ct_file, seg_file):
    # load files
    ct_img = tio.ScalarImage(ct_file)
    seg_label = tio.LabelMap(seg_file)
    # print(torch.max(ct_img.data), torch.max(seg_label.data))

    # standardize spacing
    return self.transform(ct_img).data, self.transform(seg_label).data


  def get_2d_bb_indices(self, seg_label):
    # find max indices of segment mask
    # ax2_min, ax1_min = torch.min(torch.nonzero(seg_label.data), axis=0).values
    # ax2_max, ax1_max = torch.max(torch.nonzero(seg_label.data), axis=0).values

    # check if no positive labels in slice
    if torch.nonzero(seg_label).numel() == 0:
      return None

    # get connected components and bb labels
    props = regionprops_table(seg_label.numpy(), properties=('label', 'bbox'))
    bb_df = pd.DataFrame(props)

    bb_indices = []

    for index, row in bb_df.iterrows():
      # bbox-0 to bbox-3: (min_row, min_col, max_row, max_col)
      # Pixels belonging to the bounding box are in the half-open interval
      # [min_row; max_row) and [min_col; max_col).
      # ax1: column, ax2: row
      # want to return as ax1_min, ax2_min, ax1_max, ax2_max
      bb_indices.append([row['bbox-1'], row['bbox-0'], row['bbox-3'], row['bbox-2']])

    return bb_indices


  def get_3d_bb_indices(self, seg_label):
    # find max indices of segment mask
    _, z_min, y_min, x_min = torch.min(torch.nonzero(seg_label.data), axis=0).values
    _, z_max, y_max, x_max = torch.max(torch.nonzero(seg_label.data), axis=0).values

    return [z_min, z_max, y_min, y_max, x_min, x_max]


  def extend_bb_label(self, seg_label):
    # find max indices of segment mask
    z_min, z_max, y_min, y_max, x_min, x_max = self.get_bb_indices(seg_label)

    # set voxels in 3D bb to 1
    bb_label = seg_label.tensor.clone()
    bb_label[0, z_min:z_max+1, y_min:y_max+1, x_min:x_max+1] = 1

    return bb_label


  # def get_scaled_slice(self, ct_tensor_slice):
  #   # scale image to 255 using max-min
  #   scaled_slice = (ct_tensor_slice - torch.min(ct_tensor_slice)) / \
  #                  (torch.max(ct_tensor_slice) - torch.min(ct_tensor_slice) + 1e-6) * 255
  #   scaled_slice = scaled_slice.clamp(0, 255)

  #   # scaled_slice = ct_tensor_slice

  #   return scaled_slice


  def process_three_channels(self, ct_img_permuted, a_i):
    axis_max = ct_img_permuted.shape[0]

    # skip if lower/upper slice out of bound
    if a_i == 0 or a_i == axis_max - 1:
      return None

    return ct_img_permuted[a_i-1:a_i+2, :, :]

  def count_mask_area(self, seg_label):
    return torch.count_nonzero(seg_label.data)

  def split_bb(self, mask, max_bb = 5):
    # IOU track
    max_IOU = 0

    # current area ratio
    x_max, x_min, y_max, y_min = -1,len(mask[0]),-1,len(mask[0])
    mask_count = 0
    for row in range(0, len(mask)):
      for col in range(0, len(mask[0])):
        if(mask[row, col] == 1):
          y_min = min(y_min, row)
          y_max = max(y_max, row)
          x_min = min(x_min, col)
          x_max = max(x_max, col)
          mask_count += 1

    # processing mask
    data_points = []
    for row in range(0, len(mask)):
      for col in range(0, len(mask[0])):
        if(mask[row, col] == 1):
          data_points.append([row, col])
    data_points = np.array(data_points)
    data_points = np.float32(data_points)


    # k-means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    for number_of_bb in range(1,max_bb+1):
      ret,label,center=cv2.kmeans(data_points,number_of_bb,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
      bb_boundary_points = []
      for i in range(0, number_of_bb):
        bb_boundary_points.append([len(mask[0]),-1, len(mask[0]), -1])
      for i in range(0, len(data_points)):
        curr_point = data_points[i]
        curr_cluster = int(label[i])
        bb_boundary_points[curr_cluster][0] = int(min(bb_boundary_points[curr_cluster][0], curr_point[0]))
        bb_boundary_points[curr_cluster][1] = int(max(bb_boundary_points[curr_cluster][1], curr_point[0]))
        bb_boundary_points[curr_cluster][2] = int(min(bb_boundary_points[curr_cluster][2], curr_point[1]))
        bb_boundary_points[curr_cluster][3] = int(max(bb_boundary_points[curr_cluster][3], curr_point[1]))

      bb_mask = np.zeros((mask.shape[0], mask.shape[1]))
      total_bb_area = 0
      for bb in bb_boundary_points:
        y_min, y_max, x_min, x_max = bb
        for row in range(int(y_min), int(y_max)+1):
          for col in range(int(x_min),int(x_max)+1):
            bb_mask[row, col] = 1
        #total_bb_area += (bb[0]-bb[1])*(bb[2]-bb[3])

      #print(np.unique(bb_mask))
      for row in range(0, len(bb_mask)):
        for col in range(0, len(bb_mask[0])):
          if(bb_mask[row, col] == 1):
            total_bb_area += 1

      if(mask_count/total_bb_area > max_IOU):
        bb_return = bb_boundary_points

      #print("Area ratio (mask to bb) for {} clusters: ".format(number_of_bb), mask_count/total_bb_area )
    return bb_return


  def run(self):
    # get list of subjects
    ct_nii_files = self.find_ct_files()

    # load SUV image
    for seg_file, ct_file in tqdm(ct_nii_files):
      # swap file names if needed
      if 'SUV' in str(seg_file):
        seg_file, ct_file = ct_file, seg_file

      # get patient ID
      patient_id = seg_file.parts[-3]

      # load nii files
      ct_img, seg_label = self.load_and_standardize_spacing(ct_file, seg_file)
      # bb_indices = self.get_3d_bb_indices(seg_label)   # z_min, z_max, y_min, y_max, x_min, x_max

      # convert each slice to PIL Image and save
      ct_img = ct_img.squeeze() # convert to C x H x W from B x C x H x W
      seg_label = seg_label.squeeze()
      # print(ct_img.shape)

      # permute image by axis
      if self.axis == 0:
        # sagittal plane
        permute_axis = (0, 1, 2)
      elif self.axis == 1:
        # coronal plane
        permute_axis = (1, 0, 2)
      elif self.axis == 2:
        # axial plane
        permute_axis = (2, 0, 1)

      ct_img_permuted = torch.permute(ct_img, permute_axis)
      seg_label_permuted = torch.permute(seg_label, permute_axis)

      # Faster-RCNN does not support images without detections
      # for a_i in range(bb_indices[self.axis*2], bb_indices[self.axis*2+1]):
      ax_max_idx = ct_img_permuted.shape[0]
      for a_i in range(0, ax_max_idx):
        ct_tensor_slice = ct_img_permuted[a_i, :, :]
        seg_label_slice = seg_label_permuted[a_i, :, :]

        # get bb indices
        slice_bb_indices = self.get_2d_bb_indices(seg_label_slice)

        if not self.save_negatives and not slice_bb_indices:
          # skip if not saving negatives or no lesion in image
          # print(slice_bb_indices)
          continue

        if self.channels == 3:
          ct_tensor_slice = self.process_three_channels(ct_img_permuted, a_i)
          # print(ct_tensor_slice.shape)

        if ct_tensor_slice is None:
          continue

        # save image
        img_file_name = f'{patient_id}_{self.plane}_{a_i:0>3}.jpg'
        img_path = self.output_folder / img_file_name

        kwargs = {'normalize': True}
        #save_image(ct_tensor_slice, img_path, **kwargs)

        if slice_bb_indices:
          # create csv label
          with open(self.label_file, 'a') as op:
            if os.stat(self.label_file).st_size == 0:
              op.write(f'img_filename{self.delim}x_min{self.delim}y_min{self.delim}x_max{self.delim}y_max{self.delim}cancer_type{self.delim}img_width{self.delim}img_height\n')

            for ax1_min, ax2_min, ax1_max, ax2_max in slice_bb_indices:
              threshold = 0.1
              bb_area = (abs(ax1_min-ax1_max)*abs(ax2_min-ax2_max))
              mask_area = self.count_mask_area(seg_label_slice).item()

              if(mask_area/bb_area < threshold and mask_area > 100):
                bb_boxes = self.split_bb(seg_label_slice.numpy())
                for y_min, y_max, x_min, x_max in bb_boxes:
                  with open(self.label_file, 'a') as op:
                    op.write(
                      (f'{img_file_name}{self.delim}'
                      f'{x_min}{self.delim}{y_min}{self.delim}'
                      f'{x_max}{self.delim}{y_max}{self.delim}'
                      f'{self.label}{self.delim}{ct_tensor_slice.shape[1]}{self.delim}{ct_tensor_slice.shape[2]}\n')
                )
              else:
                with open(self.label_file, 'a') as op:
                  op.write(
                      (f'{img_file_name}{self.delim}'
                      f'{ax1_min}{self.delim}{ax2_min}{self.delim}'
                      f'{ax1_max}{self.delim}{ax2_max}{self.delim}'
                      f'{self.label}{self.delim}{ct_tensor_slice.shape[1]}{self.delim}{ct_tensor_slice.shape[2]}\n')
                )

In [ ]:
import pandas as pd
import matplotlib.patches as patches

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/Shared_csv/all_patients.csv')

def get_label(id):
  return (df[df['Subject ID']==id]['diagnosis'].values[0]).lower()

In [ ]:
# split into 10 parts to
nifti_folders = os.listdir('/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/data/nifti')
nifti_folders = [nifti_folders[i:i + 90] for i in range(0, len(nifti_folders), 90)]
print(len(nifti_folders), len(nifti_folders[0]))

for i in range(len(nifti_folders)):
  print(len(nifti_folders[i]))

10 90
90
90
90
90
90
90
90
90
90
90


In [ ]:
#
for count, f in enumerate(nifti_folders[9]):
  if (count+1) % 10 == 0:
    print(count+1)
  label = get_label(f)
  if os.path.isdir(f'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means2/SUV_{label}_k_means/{f}')==True:
    shutil.rmtree(f'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means2/SUV_{label}_k_means/{f}')
  os.makedirs(f'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means2/SUV_{label}_k_means/{f}')
  DataProcessor = TwoDimensionSlicesProcessing(nifti_folder=f'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/data/nifti/{f}',
                                              output_folder=f'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means2/SUV_{label}_k_means',
                                              label=label,
                                              label_file=f'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means2/SUV_{label}_k_means/{f}/labels.csv',
                                              channels=3,
                                              save_negatives=False)
  DataProcessor.run()

In [ ]:
print("part 9 above cell done")

part 9 above cell done


In [ ]:
/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/SUV_melanoma_k_means/PETCT_7323c415d0/labels.csv

# Run the below cell only for k-means data

In [ ]:
df = pd.DataFrame()
from os import walk
total_rows = 0

for i in ['SUV_lung_cancer','SUV_lymphoma','SUV_melanoma']:
  folder = f'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means2/{i}_k_means'
  for (dirpath, dirnames, filenames) in walk(folder):
    for s in dirnames:
      subfolder = dirpath + '/' + s
      tempdf = pd.read_csv(subfolder + '/labels.csv')
      index_to_change = tempdf.index[tempdf[tempdf.columns[1]] == 'x_min'].tolist()
      index_to_change = index_to_change[0]
      tempdf.iloc[index_to_change] = tempdf.columns
      tempdf.columns = ['img_filename',	'x_min',	'y_min',	'x_max',	'y_max',	'cancer_type',	'img_width',	'img_height']
      total_rows += tempdf.shape[0]
      df = pd.concat([df, tempdf], axis=0)

# df = df[['img_filename','x_max','x_min','y_max','y_min', 'cancer_type', 'img_width', 'img_height']]
# df.columns = ['img_filename',	'x_min','y_min','x_max','y_max','cancer_type','img_width','img_height']

df[['x_min', "y_min", "x_max", "y_max"]] = df[["x_min", "y_min", "x_max", "y_max"]].apply(pd.to_numeric)

In [ ]:
print(total_rows)

print(df.shape)

print(df.columns)

55070
(55070, 8)
Index(['img_filename', 'x_min', 'y_min', 'x_max', 'y_max', 'cancer_type',
       'img_width', 'img_height'],
      dtype='object')


In [ ]:
count = 0
for i in range(0, len(df)):
  data = df.iloc[i]
  if( (data['x_min'] > data['x_max']) or (data['y_min'] > data['y_max']) ):
    count += 1

print(count, len(df))

0 55070


In [ ]:
data

img_filename    PETCT_f37014ec85_axial_549.jpg
x_min                                      216
y_min                                    161.0
x_max                                    231.0
y_max                                    173.0
cancer_type                           melanoma
img_width                                  408
img_height                                 408
Name: 23, dtype: object

In [ ]:
print("done")

done


In [ ]:
# df = pd.DataFrame()
# from os import walk

# for i in ['CT_lung_cancer','CT_lymphoma','CT_melanoma']:
#   folder = f'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/{i}_k_means'
#   for (dirpath, dirnames, filenames) in walk(folder):
#     for s in dirnames:
#       subfolder = dirpath + '/' + s
#       tempdf = pd.read_csv(subfolder + '/labels.csv')
#       df = pd.concat([df, tempdf], axis=0)

In [ ]:
df.to_csv('/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means2/SUV_labels.csv', index=False)

In [ ]:
import pandas as pd
a = pd.read_csv('/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/Shared_csv/all_patients.csv')
# a[['Subject ID','diagnosis']]
a.drop_duplicates(subset='Subject ID')[['Subject ID','diagnosis']].groupby('diagnosis').count()

,Subject ID
diagnosis,
LUNG_CANCER,168
LYMPHOMA,137
MELANOMA,169
NEGATIVE,426


In [ ]:
proportions = [168/900, 137/900, 169/900, 426/900]
proportions

[0.18666666666666668,
 0.15222222222222223,
 0.18777777777777777,
 0.47333333333333333]